# Development of Wastewater Surveillance Data Automation Script 

## 1) Export All data from LIMS DatabasE

In [3]:
#run script that executes export of LIMS data

%run -i "viral_lims_export.py"

In [ ]:
df = export_df_from_LIMS()
df.info()

# Appendix 

## A) Explore datatype stored in LIMS database

pyodbc cursor object allows to interact with database parameters. cursos.columns() returns information about every column in the database table.

In [ ]:
cnxn = pyodbc.connect(credentials) # credentials = 'DSN=LIMS_DATA;UID=xxxxxxx;PWD=xxxxxxx'
cursor = cnxn.cursor()

dtype_list = [(i.column_name, i.type_name) for i in cursor.columns(table="vz_Epi_ELS_SARS-CoV-2 ddPCR")]

dtype_list


RESULT: Two columns have datetime type, remaining columns are varchar type
('TestResultDate', 'datetime')
('SampleCollectDate', 'datetime')

## B) Explore converting LIMS dataframe to numeric type - may not be necessary.

In [52]:
potential_numeric = ["NumNoTargetControl", "SARSCoV2AvgConc"]

In [55]:
df[potential_numeric] = df[potential_numeric].apply(pd.to_numeric, errors = "coerce")

In [ ]:
df.info()